In [1]:
!pip install rlcard more_itertools 

You should consider upgrading via the 'c:\users\bautista\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import random
random.seed(69)

In [4]:
card_tiers = [
        (['E1'], 13),
        (['B1'], 12),
        (['E7'], 11),
        (['O7'], 10),
        (['E3', 'B3', 'O3', 'C3'], 9),
        (['E2', 'B2', 'O2', 'C2'], 8),
        (['O1', 'C1'], 7),
        (['E12', 'B12', 'O12', 'C12'], 6),
        (['E11', 'B11', 'O11', 'C11'], 5),
        (['E10', 'B10', 'O10', 'C10'], 4),
        (['C7', 'B7'], 3),
        (['E6', 'B6', 'O6', 'C6'], 2),
        (['E5', 'B5', 'O5', 'C6'], 1),
        (['E4', 'B4', 'O4', 'C4'], 0)
    ]

''' Game-related base classes
'''
class Card:
    '''
    Card stores the suit and rank of a single card
    Note:
        The suit variable in a standard card game should be one of [C, B, E, O] meaning [Copa, Basto, Espada, Oro]
        Similarly the rank variable should be one of ['1', '2', '3', '4', '5', '6', '7', '10', '11', '12']
    '''
    suit = None
    rank = None
    valid_suit = ['C', 'B', 'E', 'O']
    valid_rank = ['1', '2', '3', '4', '5', '6', '7', '10', '11', '12']
    

    def __init__(self, suit, rank):
        ''' Initialize the suit and rank of a card
        Args:
            suit: string, suit of the card, should be one of valid_suit
            rank: string, rank of the card, should be one of valid_rank
        '''
        self.suit = suit
        self.rank = rank
        for cards, tier in card_tiers:
            if suit+rank in cards:
                self.tier = tier
                break
        

    def __eq__(self, other):
        if isinstance(other, Card):
            return self.rank == other.rank and self.suit == other.suit
        else:
            # don't attempt to compare against unrelated types
            return NotImplemented

    def __hash__(self):
        suit_index = Card.valid_suit.index(self.suit)
        rank_index = Card.valid_rank.index(self.rank)
        return rank_index + 100 * suit_index

    def __str__(self):
        ''' Get string representation of a card.
        Returns:
            string: the combination of rank and suit of a card. Eg: AS, 5H, JD, 3C, ...
        '''
        return self.get_index()
    
    def compare_as_str(self, card_as_string):
        return card_as_string == self.__str__()
    
    def get_index(self):
        ''' Get index of a card.
        Returns:
            string: the combination of suit and rank of a card. Eg: 1S, 2H, AD, BJ, RJ...
        '''
        return self.suit+self.rank

In [5]:
def init_truco_deck():
    ''' Initialize a truco deck of 52 cards
    Returns:
        (list): A list of Card object
    '''
    suit_list = ['C', 'B', 'E', 'O']
    rank_list = ['1', '2', '3', '4', '5', '6', '7', '10', '11', '12']
    res = [Card(suit, rank) for suit in suit_list for rank in rank_list]
    return res

def shuffle_cards(cards, in_place=False):
    import random
    if in_place:
        random.shuffle(cards)
        return cards
    else:
        import copy 
        copied_cards = copy.deepcopy(cards)
        random.shuffle(copied_cards)
        return copied_cards

In [6]:
class Player:
    
    def __init__(self, player_id):
        ''' Initilize a player.
        Args:
            player_id (int): The id of the player
        '''
        self.player_id = player_id
        self.hand = []
        
    def __str__(self):
        return f"Player {self.player_id} | Cards: {[str(c) for c in self.hand]}"
    
    def __eq__(self, other):
        if isinstance(other, Player):
            return self.get_id() == other.get_id()
        return False

    def get_id(self):
        ''' Return the id of the player
        '''

        return self.player_id

In [7]:
class Dealer:
    
    def __init__(self):
        '''
        Initialize a dealer.
        '''
        self.deck = init_truco_deck()
        shuffle_cards(self.deck, True)
        
    def deal_cards_in_order(self, players, amount=3):
        player_index = 0
        for _ in range(amount * len(players)):
            if player_index == len(players):
                player_index = 0
            players[player_index].hand.append(self.deck.pop())
            player_index+=1


In [8]:
import numpy as np


truco_actions = np.array([ 
    'truco', 
    're-truco', 
    'vale cuatro'
])

envido_actions = np.array([ 
    'envido',
    'real envido',
])

response_actions = np.array([ 
    'quiero',
    'no quiero',
])

other_actions = np.array([
    'fold'
])

playable_actions = np.concatenate((truco_actions, envido_actions, response_actions, other_actions))


playable_cards = np.array([str(c) for c in init_truco_deck()])

game_actions = np.concatenate((playable_actions, playable_cards))

In [12]:
cards1 = [Card('C', '7'), Card('O', '7'), Card('C', '10')] 
cards2 = [Card('B', '7'), Card('O', '7'), Card('C', '10')]
cards3 = [Card('O', '7'), Card('C', '11'), Card('C', '10')]
cards4 = [Card('B', '7'), Card('B', '5'), Card('C', '10')]
cards5 = [Card('B', '7'), Card('O', '5'), Card('C', '3')] 
cards6 = [Card('C', '11'), Card('C','5'), Card('C', '1')] 
cards7 = [Card('C', '11'), Card('C','10'), Card('C', '1')] 
cards8 = [Card('C', '11'), Card('C','7'), Card('C', '1')] 
cards9 = [Card('C', '11'), Card('C','10'), Card('C', '12')] 
cards10 = [Card('B', '7'), Card('C','10'), Card('E', '12')] 

def test(cards, expected):
    result = calculate_envido(cards)
    if result == expected:
        print(True)
    else:
        print(f"expected {expected} but got {result}")

test(cards1, 27)
test(cards2, 7)
test(cards3, 20)
test(cards4, 32)
test(cards5, 7)
test(cards6, 26)
test(cards7, 21)
test(cards8, 28)
test(cards9, 20)
test(cards10, 7)

True
True
True
True
True
True
True
True
True
True


In [11]:
from more_itertools import locate
import numbers

envido_states = [
    (['envido', 'quiero'], 2),
    (['envido', 'no quiero'], 1),
    (['envido','envido', 'quiero'], 4),
    (['envido','envido', 'no quiero'], 2),
   (['envido', 'real envido', 'quiero'], 5),
    (['envido', 'real envido', 'no quiero'], 2),
    (['real envido', 'quiero'], 3),
    (['real envido', 'no quiero'], 1)
]


truco_states = [
    (['truco', 'quiero'], 2),
    (['truco', 'no quiero'], 1),
   (['truco', 're-truco', 'quiero'], 3),
   (['truco', 're-truco', 'no quiero'], 2),
    (['truco', 're-truco', 'vale cuatro', 'quiero'], 4),
    (['truco', 're-truco', 'vale cuatro', 'no quiero'], 3),
    # double downs
   (['truco', 'quiero','re-truco', 'quiero'], 3),
    (['truco', 'quiero','re-truco', 'no quiero'], 2),
    (['truco', 're-truco', 'quiero', 'vale cuatro', 'quiero'], 4),
    (['truco', 're-truco', 'quiero', 'vale cuatro', 'no quiero'], 3),
    (['truco', 'quiero', 're-truco', 'vale cuatro', 'quiero'], 4),
    (['truco', 'quiero', 're-truco', 'vale cuatro', 'no quiero'], 3),
]



def is_valid_state(all_states, current_state, action):
    raw_state = np.append([call for player, call in current_state], action)
    
    for state, score in all_states:
        zipped = list(zip(state, raw_state))
        if len(zipped) != len(raw_state):
            continue
        if np.all([True if a == b else False for a, b in zipped]):
            return True
    return False

def is_wager_finished(current_state):
    return len(current_state) > 0 and current_state[-1][1] == 'quiero' or current_state[-1][1] ==  'no quiero'

def is_wager_started(current_state):
    return len(current_state) > 0;

def is_wager_active(current_state):
    return is_wager_started(current_state) and not is_wager_finished(current_state)

def get_wager_reward(all_states, current_state):
    raw_state = [call for player, call in current_state]
    for state, score in all_states:
        if len(state) == len(raw_state) and np.all(state == raw_state):
            return score
    raise Exception(f"Can't get wager reward. No wager matching {raw_state}")
    
    
def rank_to_envido_value(rank):
    if rank in ['10', '11', '12']:
        return 10;
    else:
        return int(rank)
    
def calculate_envido(cards):
    groups = {}
    
    for card in cards:
        value = rank_to_envido_value(card.rank)
        if card.suit in groups:
            groups[card.suit].append(value)
        else:
            groups[card.suit] = [value]
    
    score = 0
    for key, values in groups.items():
        if len(values) == 1:
            score = max(score, values[0] if values[0] != 10 else 0)
        elif len(values) == 2:
            #list with no duplicates
            new_values = list(dict.fromkeys(values))
            # double face card
            if len(new_values) < len(values):
                score = max(score, sum(values))
            elif 10 in new_values:
                score = max(score, sum(new_values) + 10)
            else:
                score = max(score, sum(new_values) + 20)
        else:
            #list with no duplicates
            new_values = list(dict.fromkeys(values))
            if len(new_values) == 1:
                score = max(score, 20)
            elif len(new_values) == 2:
                score = max(score, sum(new_values) + 10)
            elif 10 in new_values:
                score = max(score, sum(new_values) + 10)
            else:
                two_largest = np.argsort(new_values)
                score = max(score, sum(two_largest[-2:]))
                
    return score
            

class TrucoHand:
    
    
    def __init__(self, players, goes_first=0):
        '''
        Initialize a Hand of Truco.
        '''
        self.players = players
        self.finished = False
        self.round = 0
        self.truco_next = None
        self.envido_next = None
        self.has_retruco = None
        self.card_next = self.players[goes_first]
        self.cards_played = []
        self.truco_calls = []
        self.envido_calls = []
        self.scoreboard = np.array([(p, 0) for p in players])
        self.first_move_by = self.players[goes_first] 
        self.second_move_by = self.players[1 - goes_first] 
        
    def get_envido_winner(self):    
        p1_cards = np.concatenate(([card for player, card in self.cards_played if player == self.first_move_by], self.first_move_by.hand))
        p2_cards = np.concatenate(([card for player, card in self.cards_played if player == self.second_move_by], self.second_move_by.hand))
        
        p1_envido = calculate_envido(p1_cards)
        p2_envido = calculate_envido(p2_cards)
        
        print(f"{self.first_move_by} has an envido of {p1_envido}")
        print(f"{self.second_move_by} has an envido of {p2_envido}")
        
        return self.first_move_by if p1_envido >= p2_envido else self.second_move_by
    
    def get_card_winner(self):
        p1_cards = [card for player, card in self.cards_played if player == self.first_move_by]
        p2_cards = [card for player, card in self.cards_played if player == self.second_move_by]
        
        p1_wins=0
        p2_wins=0
        next_wins = False
        for c1, c2 in zip(p1_cards, p2_cards):
            comparison = c1.tier - c2.tier
            if comparison > 0:
                p1_wins += 1
                if next_wins:
                    break
            elif comparison < 0:
                p2_wins += 1
                if next_wins:
                    break
            elif comparison == 0:
                if p1_wins > p2_wins:
                    p1_wins += 1
                    break;
                elif p2_wins > p1_wins:
                    p2_wins += 1
                    break;
                else:
                    next_wins = True
                
        if p1_wins < 2 and p2_wins < 2:
            return None
        
        return self.first_move_by if p1_wins >= p2_wins or (p1_wins == 0 and p2_wins == 0 and next_wins) else self.second_move_by

    def update_score(self, player, score):
        if player == self.scoreboard[0][0]:
            self.scoreboard[0][1] += score
        else:
            self.scoreboard[1][1] += score
    
    def get_envido_reward(self):
        return get_wager_reward(envido_states, self.envido_calls)
    
    def get_truco_reward(self):
        return get_wager_reward(truco_states, self.truco_calls)
        
    def is_truco_started(self):
        return is_wager_started(self.truco_calls)

    def is_truco_active(self):
        return is_wager_active(self.truco_calls)

    def is_envido_active(self):
        return is_wager_active(self.envido_calls)


    def is_valid_envido_state(self, action):
        return is_valid_state(envido_states, self.envido_calls, action)

    def is_valid_truco_state(self, action):
        return is_valid_state(truco_states, self.truco_calls, action)
    


    def is_truco_finished(self):
        return is_wager_finished(self.truco_calls)

    def is_envido_finished(self):
        return is_wager_finished(self.envido_calls)
        
    def get_opponent(self, player):
        if player == self.players[0]:
            return self.players[1]
        else:
            return self.players[0]
        
    def switch_card_turn(self):
        self.card_next = self.get_opponent(self.card_next)
    
    def switch_envido_turn(self):
        self.envido_next = self.get_opponent(self.envido_next)
        
    def switch_truco_turn(self):
        self.truco_next = self.get_opponent(self.truco_next)
        
    def finish_hand(self):
        self.finished = True
        print("Hand finished.")
        print(f"{self.scoreboard[0][0]} scored {self.scoreboard[0][1]}")
        print(f"{self.scoreboard[1][0]} scored {self.scoreboard[1][1]}")
    
    def finish_round(self):
        self.round += 1
        
        first_played = self.cards_played[-2]
        second_played = self.cards_played[-1]
        comparison = first_played[1].tier - second_played[1].tier
        if comparison >= 0:
            self.switch_card_turn() # switch if second person wins round
            print(f"{self.card_next} won the round. They will start the next.")
            
        print("Round finished")
        
        winner = self.get_card_winner()
        if winner is not None: 
            if self.is_truco_started():
                reward = self.get_truco_reward()
                self.update_score(winner, reward)
                print(f"{winner} was rewarded {reward} for winning truco.")
            else:
                self.update_score(winner, 1)
                print(f"{winner} was rewarded 1 for winning hand.")
            self.finish_hand()
        
    def take_action(self, action):
        player = self.card_next
        if self.envido_next is not None:
            player = self.envido_next
        elif self.truco_next is not None:
            player = self.truco_next
            
        action_played = game_actions[action] if isinstance(action, numbers.Number) else action
        
        if action_played in envido_actions:
            if self.round == 0:
                if not self.is_truco_active():
                    if len(self.envido_calls) == 0:
                        self.envido_calls.append((player, action_played))
                        print(f"{player} called {action_played}.")
                        self.envido_next = self.get_opponent(player)
                    elif self.envido_calls[-1][0] != player and self.is_valid_envido_state(action_played):
                        self.envido_calls.append((player, action_played))
                        print(f"{player} called {action_played}")
                        self.switch_envido_turn()
                    else:
                        print(f"{player} can't call {action_played}.")
                else:
                    print(f"{player}: Envido can only be played before Truco.")
            else:
                print(f"{player}: Envido can only be played in the first round")
        elif action_played in truco_actions:
            if not self.is_envido_active():
                if not self.is_truco_started() and action_played == "truco":
                    self.truco_calls.append((player,action_played))
                    print(f"{player} called {action_played}")
                    self.truco_next = self.get_opponent(player)
                elif self.is_truco_started() and self.is_valid_truco_state(action_played):
                    if self.is_truco_active() and self.truco_next == player:
                        self.truco_calls.append((player, action_played))
                        print(f"{player} called {action_played}")
                        self.truco_next = self.get_opponent(player)
                    elif self.has_retruco == player:
                        self.truco_calls.append((player, action_played))
                        print(f"{player} called {action_played}")
                        self.truco_next = self.get_opponent(player)
                        self.has_retruco = self.truco_next 
                    else:
                        print(f"{player} can't call {action_played}.")
                else:
                    print(f"{player} can't call {action_played}.")
            else:
                print(f"{player} can't call {action_played} unless envido has finished.")
        elif action_played in response_actions:
            if self.is_envido_active() and self.is_valid_envido_state(action_played):
                self.envido_calls.append((player, action_played))
                print(f"{player} called {action_played} envido")
                if action_played == "no quiero":
                    opponent = self.get_opponent(player)
                    reward = self.get_envido_reward()
                    self.update_score(opponent, reward)
                    print(f"{opponent} was rewarded {reward} for winning envido.")
                else:
                    winner = self.get_envido_winner()
                    reward = self.get_envido_reward()
                    self.update_score(winner, reward)
                    print(f"{winner} was rewarded {reward} for winning envido.")
                # de-activate envido    
                self.envido_next = None
            elif self.is_truco_active() and self.is_valid_truco_state(action_played):
                self.truco_calls.append((player, action_played))
                print(f"{player} called {action_played} truco")
                if action_played == "no quiero":
                    opponent = self.get_opponent(player)
                    reward = self.get_truco_reward()
                    self.update_score(opponent, reward)
                    print(f"{opponent} was rewarded {reward} for winning truco.")
                    self.finish_hand()
                else:
                    #No response needed
                    self.has_retruco = player
                    self.truco_next = None
            else:
                print(f"{player} can't call {action_played} right now.")
        elif action_played in playable_cards:
            if not self.is_envido_active():
                if not self.is_truco_active():
                    card_played_indexes = list(locate(player.hand, lambda c: c.compare_as_str(action_played)))
                    if len(card_played_indexes) == 1:
                        card_played = player.hand.pop(card_played_indexes[0])
                        self.cards_played.append((player, card_played))
                        print(f"{player} played {card_played}")
                        if len(self.cards_played) % 2 == 0:
                            self.finish_round()
                        else:
                            self.switch_card_turn()
                    elif len(card_played_indexes) > 1:
                        print(f"{player}: card_played_indexes should never be > 1. Current value: {card_played_indexes}")
                    else:
                        print(f"{player} can't play the card {action_played}. They don't have it.")
                else:
                    print(f"{player} can't play the card {action_played} before responding to truco.")
            else:
                print(f"{player} can't play the card {action_played} before responding to envido.")
        elif action_played == "fold":
            print(f"{player} folded.")
            if self.is_envido_active():
                self.envido_calls.append((player, 'no quiero'))
                print(f"{player} forfeited envido")
                opponent = self.get_opponent(player)
                reward = self.get_envido_reward()
                self.update_score(opponent, reward)
                print(f"{opponent} was rewarded {reward} for winning envido.")
            if self.is_truco_started():
                if self.is_truco_active():
                    self.truco_calls.append((player, 'no quiero'))
                print(f"{player} forfeited truco")
                opponent = self.get_opponent(player)
                reward = self.get_truco_reward()
                self.update_score(opponent, reward)
                print(f"{opponent} was rewarded {reward} for winning truco.")
            else:
                opponent = self.get_opponent(player)
                self.update_score(opponent, 1)
                print(f"{opponent} was rewarded 1 for winning hand.")
            self.finish_hand()
            
        
            
    

In [430]:
num_players = 2

players = [Player(id) for id in range(num_players)]

dealer = Dealer()
dealer.deal_cards_in_order(players)

hand = TrucoHand(players)

In [431]:
for p in players:
    print(p)

Player 0 | Cards: ['B4', 'O7', 'C10']
Player 1 | Cards: ['E6', 'O6', 'C6']


In [432]:
hand.take_action('truco')

Player 0 | Cards: ['B4', 'O7', 'C10'] called truco


In [437]:
hand.take_action('quiero')

Player 0 | Cards: ['B4', 'C10'] called quiero truco


In [ ]:
hand.take_action('envido')

In [ ]:
hand.take_action('real envido')

In [ ]:
hand.take_action('no quiero')

In [436]:
hand.take_action('re-truco')

Player 1 | Cards: ['E6', 'O6', 'C6'] called re-truco


In [435]:
hand.take_action('O7')

Player 0 | Cards: ['B4', 'C10'] played O7


In [438]:
hand.take_action('E6')

Player 1 | Cards: ['O6', 'C6'] played E6
Player 0 | Cards: ['B4', 'C10'] won the round. They will start the next.
Round finished


In [439]:
hand.take_action('C10')

Player 0 | Cards: ['B4'] played C10


In [440]:
hand.take_action('C6')

Player 1 | Cards: ['O6'] played C6
Player 0 | Cards: ['B4'] won the round. They will start the next.
Round finished
Player 0 | Cards: ['B4'] was rewarded 3 for winning truco.
Hand finished.
Player 0 | Cards: ['B4'] scored 3
Player 1 | Cards: ['O6'] scored 0


In [ ]:
hand.take_action('O1')

In [ ]:
hand.take_action('B11')

In [ ]:
hand.take_action('fold')